# Model Server

## nuclio code section

In [1]:
# nuclio: ignore
import nuclio

Install the following packages available to the function:

In [2]:
%%nuclio cmd -c
pip uninstall -y mlrun
pip install -U -q git+https://github.com/mlrun/mlrun.git@development
pip install -U -q kfserving
pip install -U -q azure
pip install -U -q numpy==1.17.4
pip install -U -q lightgbm

In [3]:
import kfserving
import os
import numpy as np
from pickle import load
from typing import List
import lightgbm as lgb

In [4]:
TARGET_PATH = '/User/mlrun/lightgbm'
MODEL_NAME = 'lightgbm_classifier.pkl'

In [24]:
class MyLGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: lgb.LGBMClassifier = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if booster:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        model_file = os.path.join(TARGET_PATH, MODEL_NAME)
        self.classifier = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body:  A list of observations, each of which is an 1-dimensional 
                      feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` 
        input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [25]:
# nuclio: end-code

____

## **deploy**

In [26]:
from mlrun import new_model_server, mount_v3io

In [27]:
fn = new_model_server('lgbm_inference_model', 
                      models={'lgbm_pickle': TARGET_PATH}, 
                      model_class='MyLGBoostModel').apply(mount_v3io())

In [28]:
#fn.spec.no_cache=True

In [29]:
addr = fn.deploy(project='mlrun-demos')

[mlrun] 2020-01-15 11:22:42,475 deploy started
[nuclio] 2020-01-15 11:22:49,613 (info) Build complete
[nuclio] 2020-01-15 11:22:55,707 (info) Function deploy complete
[nuclio] 2020-01-15 11:22:55,713 done updating lgbm-inference-model, function address: 3.135.246.153:32716


<a id="deploy"></a>
_______________________________________________________________________________

## **test**

In [30]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import json
import os

import requests

#### grab 3 rows of data

In [31]:
# Grab some data fromthe test set
features = pq.read_table(os.path.join(TARGET_PATH, 'xtest.parquet')).to_pandas().iloc[:3, :]
labels = pq.read_table(os.path.join(TARGET_PATH, 'ytest.parquet')).to_pandas().iloc[:3, :]

#### create an event and wrap it in json

In [32]:
event = {"instances": features.values.tolist()}

If the notebook is restarted for some reason, however the function has already been deployed, simply uncomment the following cell and paste in the original endpoint here.  You can 
retrieve the function's enpoint address from the platform ui under **Projects**. Look for
the project name you gave to the deployment of interest, in our case **mlrun-demos**.
Click on the function in the project, copy the 'Invocation URL' and paste here:

In [33]:
# addr = "3.135.246.153:32716"

In [34]:
resp = requests.post(addr + '/lgbm_pickle/predict', json=event)

In [35]:
json.loads(resp.content)

[1.0, 1.0, 0.0]

### Test of Estimated Model Object 

Here we simply grab the estimated model file created on the **[kubeflow pipeline](kubeflow%20pipeline.ipynb)**, load it, and run a test matrix through it.

In [36]:
import sklearn.preprocessing

model_file = os.path.join(TARGET_PATH, MODEL_NAME)
model_file

'/User/mlrun/lightgbm/lightgbm_classifier.pkl'

In [37]:
lgbm_model = load(open(model_file, 'rb'))

In [38]:
testvals = np.asarray(features.values)

In [39]:
lgbm_model.predict(testvals)

array([1., 1., 0.])